# Chat Bot Libraries & Data

In [1]:
import nltk
import json
import pickle
import numpy as np
import random
import keras
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mallickd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mallickd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Initializing Chatbot Training

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

# Leading Through Intents And Lemmatization

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# print (len(documents), "documents")
# print (len(classes), "classes", classes)
# print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'), 0)
pickle.dump(classes,open('classes.pkl','wb'), 0)

# Building the Deep Learning Model

## Initializing training data

In [4]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-4-3544dc409e53>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


## Create model - 3 layers
### First layer: 128 neurons
### Second layer: 64 neurons
### And 3rd output layer: contains number of neurons equal to number of intents to predict output intent with softmax

In [5]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2165 - accuracy: 0.1064
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1445 - accuracy: 0.1489
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0929 - accuracy: 0.2340
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9424 - accuracy: 0.3191
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9309 - accuracy: 0.3617
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7111 - accuracy: 0.5745
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 1.5328 - accuracy: 0.5532
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4795 - accuracy: 0.6596
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3324 - accuracy: 0.7234
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3218 - accuracy: 0.5957
Epoch 11/

10/10 [==============================] - 0s 1ms/step - loss: 0.0672 - accuracy: 0.9787
Epoch 82/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0382 - accuracy: 1.0000
Epoch 83/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0209 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0297 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0608 - accuracy: 0.9787
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0454 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0217 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0248 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0390 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0494 - accuracy: 1.0000
Epoch 91/200


10/10 [==============================] - 0s 1ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 163/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0902 - accuracy: 0.9787
Epoch 164/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0179 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0117 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0535 - accuracy: 1.0000
Epoc

# Load Model, Intents, Classes and Words 

In [6]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# All ChatBot Functions

In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

### ===================== End of Communication Chat Bot Logic =====================

# Telegram Chat Bot Libraries & Data

In [8]:
import requests
import json
import configparser as cfg

In [9]:
token="1743726854:AAH2GztAPqLNnxFQBNmMeIG0bcTuvJ6Ep_g"
base="https://api.telegram.org/bot{}/".format(token)

class bot():
    def _init_(config):
        token=read_token_from_config_file(config)
        base="https://api.telegram.org/bot{}/".format(token)
    
    def get_updates(offset=None):
        url=base + "getUpdates?timeout=10"
        if offset:
            url = url+"&offset={}".format(offset + 1)
        
        r = requests.get(url)
        return json.loads(r.content)
    
    def send_message(msg,chat_id):
        url=base + "sendMessage?chat_id={}&text={}".format(chat_id,msg)
        if msg is not None:
            requests.get(url)
            
    def make_reply(msg):
        reply=None
        if msg is not None:
            reply=chatbot_response(msg)
        return reply
    
    def read_token_from_config_file(config):
        parser = cfg.ConfigParser()
        parser.read(config)
        return parser.get('creds','token')

In [ ]:
def run_server():
    update_id=None
    while True:
        print("...")
        updates=bot.get_updates(offset=update_id)
        updates=updates["result"]
        if updates:
            for item in updates:
                update_id=item["update_id"]
                try:
                    message=item["message"]["text"]
                except:
                    message=None
                from_=item["message"]["from"]["id"]
                reply=bot.make_reply(message)
                bot.send_message(reply,from_)
                
run_server()  

...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
